<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Finance with Python

**Chapter 05 &mdash; Static Economy**

## Numerical Examples

In [ ]:
!git clone https://github.com/tpq-classes/finance_with_python.git
import sys
sys.path.append('finance_with_python')


In [ ]:
import numpy as np
np.set_printoptions(precision=5)

In [ ]:
np.random.seed(100)

In [ ]:
I = 1000

In [ ]:
S = np.random.normal(loc=100, scale=20, size=I)

In [ ]:
S[:14]

In [ ]:
S.mean()  # sample mean under equal probability measure

In [ ]:
P = np.random.random(I)

In [ ]:
P[:10]

In [ ]:
P /= P.sum()  # random probability measure

In [ ]:
P.sum()

In [ ]:
P[:10]

In [ ]:
np.dot(P, S)  # sample expected value under probability measure P

## Financial Assets

In [ ]:
M = np.array((
    (11, 25, 0,  0,  25),
    (11, 20, 30, 15, 25),
    (11, 10, 0,  20, 10),
    (11, 5,  30, 15, 0),
    (11, 0,  0,  0,  0)
))
M

In [ ]:
M0 = np.array(5 * [10.])

In [ ]:
M0

In [ ]:
M.mean(axis=0)

In [ ]:
mu = M.mean(axis=0) / M0 - 1

In [ ]:
mu

In [ ]:
(M / M0 - 1)

In [ ]:
sigma = (M / M0 - 1).std(axis=0)

In [ ]:
sigma

## Contingent Claims

In [ ]:
K = 15

In [ ]:
M[:, 1]

In [ ]:
C1 = np.maximum(M[:, 1] - K, 0)

In [ ]:
C1

In [ ]:
phi = np.linalg.solve(M, C1)

In [ ]:
phi

In [ ]:
C1 == np.dot(M, phi)

In [ ]:
C0 = np.dot(M0, phi)

In [ ]:
C0

## Market Completeness

In [ ]:
M = np.eye(5)

In [ ]:
M

In [ ]:
np.linalg.linalg.matrix_rank(M)

In [ ]:
C1 = np.arange(10, 0, -2)

In [ ]:
C1

In [ ]:
np.linalg.solve(M, C1)

In [ ]:
np.random.seed(100)

In [ ]:
M = np.random.randint(1, 10, (5, 5))

In [ ]:
M

In [ ]:
np.linalg.linalg.matrix_rank(M)

In [ ]:
np.linalg.linalg.matrix_rank(M.T)

In [ ]:
phi = np.linalg.solve(M, C1)

In [ ]:
phi

In [ ]:
np.dot(M, phi)

## Fundamental Theorems of Asset Pricing

In [ ]:
import scipy.optimize as sco  

In [ ]:
M = np.array((
    (11, 25, 0,  0,  25),
    (11, 20, 30, 15, 25),
    (11, 10, 0,  20, 10),
    (11, 5,  30, 15, 0),
    (11, 0,  0,  0,  0)
))
M

In [ ]:
np.linalg.linalg.matrix_rank(M[:, :5])

In [ ]:
M0 = np.ones(5) * 10

In [ ]:
M0

In [ ]:
r = 0.1

In [ ]:
def E(Q):
    return np.sum((np.dot(M.T, Q) - M0 * (1 + r)) ** 2) 

In [ ]:
E(np.array(5 * [0.2]))

In [ ]:
cons = ({'type': 'eq', 'fun': lambda Q: Q.sum() - 1})

In [ ]:
bnds = (5 * [(0, 1)])

In [ ]:
bnds

In [ ]:
res = sco.minimize(E, 5 * [1],
                   # method='SLSQP',
                   constraints=cons,
                   bounds=bnds)

In [ ]:
Q = res['x']

In [ ]:
Q

In [ ]:
Q.sum()

In [ ]:
np.dot(M.T, Q) / (1 + r)

In [ ]:
np.allclose(M0, np.dot(M.T, Q) / (1 + r))

## Black-Scholes-Merton Option Pricing

In [ ]:
import math

In [ ]:
S0 = 100
r = 0.05
sigma = 0.2
T = 1.0
I = 10000

In [ ]:
np.random.seed(100)

In [ ]:
ST = S0 * np.exp((r - sigma ** 2 / 2) * T +
     sigma * math.sqrt(T) * np.random.standard_normal(I))

In [ ]:
ST[:8].round(1)

In [ ]:
ST.mean()  # sample mean of simulated values

In [ ]:
S0 * math.exp(r * T)  # mathematical expectation (forward price)

In [ ]:
from pylab import mpl, plt
plt.style.use('seaborn-v0_8')
mpl.rcParams['savefig.dpi'] = 300 
mpl.rcParams['font.family'] = 'serif'

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(ST, bins=35, label='frequency');
plt.axvline(ST.mean(), color='r', label='mean')
plt.axvline(ST.mean() + ST.std(), color='y', label='sd up')
plt.axvline(ST.mean() - ST.std(), color='y', label='sd down')
plt.legend(loc=0); 

In [ ]:
K = 105

In [ ]:
CT = np.maximum(ST - K, 0)

In [ ]:
CT[:8].round(1)

In [ ]:
C0 = math.exp(-r * T) * CT.mean()  # Monte Carlo estimator for the option value

In [ ]:
C0

## Completeness of Black-Scholes-Merton

In [ ]:
B0 = 100

In [ ]:
M0 = np.array((B0, S0))

In [ ]:
BT = B0 * np.ones(len(ST)) * math.exp(r * T)

In [ ]:
BT[:4]

In [ ]:
M = np.array((BT, ST)).T

In [ ]:
M

In [ ]:
phi = np.linalg.lstsq(M, CT, rcond=None)[0]

In [ ]:
phi

In [ ]:
np.mean((np.dot(M, phi) - CT))

In [ ]:
np.dot(M0, phi)

## Merton Jump-Diffusion Option Pricing

In [ ]:
M0 = np.array((100, 100)) 

In [ ]:
r = 0.05
sigma = 0.2
lmbda = 0.3
mu = -0.3
delta = 0.1
rj = lmbda * (math.exp(mu + delta ** 2 / 2) - 1)
T = 1.0
I = 10000

In [ ]:
BT = M0[0] * np.ones(I) * math.exp(r * T)

In [ ]:
z = np.random.standard_normal((2, I))
z -= z.mean()  # 1st moment correction
z /= z.std()  # 2nd moment correction
y = np.random.poisson(lmbda, I)
y

In [ ]:
ST = S0 * (
    np.exp((r - rj - sigma ** 2 / 2) * T +
           sigma * math.sqrt(T) * z[0]) + 
    (np.exp(mu + delta * z[1]) - 1) * y
)

In [ ]:
ST.mean() * math.exp(-r * T)

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(ST, bins=35, label='frequency');
plt.axvline(ST.mean(), color='r', label='mean')
plt.axvline(ST.mean() + ST.std(), color='y', label='sd up')
plt.axvline(ST.mean() - ST.std(), color='y', label='sd down')
plt.legend(loc=0);

In [ ]:
ST = np.maximum(S0 * (
    np.exp((r - rj - sigma ** 2 / 2) * T +
           sigma * math.sqrt(T) * z[0]) + 
    (np.exp(mu + delta * z[1]) - 1) * y
), 0)

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(ST, bins=35, label='frequency')
plt.axvline(ST.mean(), color='r', label='mean')
plt.axvline(ST.mean() + ST.std(), color='y', label='sd up')
plt.axvline(ST.mean() - ST.std(), color='y', label='sd down')
plt.legend(loc=0);

In [ ]:
K = 105

In [ ]:
CT = np.maximum(ST - K, 0)

In [ ]:
C0 = math.exp(-r * T)  * np.mean(CT)  # Monte Carlo estimator for the option value

In [ ]:
C0

In [ ]:
M = np.array((BT, ST)).T

In [ ]:
phi = np.linalg.lstsq(M, CT, rcond=-1)[0]

In [ ]:
phi

In [ ]:
np.mean(np.dot(M, phi) - CT)

In [ ]:
np.dot(M0, phi)

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>